In [ ]:
from mt import *

In [ ]:
# The directory containing the images
data_dir = '/Volumes/martin/A08_A09_test'

# glob string for images
im_glob = os.path.join(data_dir, 'Round1_max_composite.tif')

# Get list of images
im_list = sorted(glob.glob(im_glob))

im_list

Each .tif file has 72 pictures. There are 6 conditions with 12 positions each. The first two area are irrelevant. For the last four, there is the A8 ABE, A8 pUC19, A9 ABE, and A9 pUC19.

There are four readout probes. Generally they correspond to 488 and 561 being for unedit barcode and 594 and 647 being for edited barcode.

In [ ]:
# Read data using skimage
im = skimage.io.imread(im_list[0])

In [ ]:
len(im)

In [ ]:
A8_ABE = im[24:36]
A8_pUC19 = im[36:48]
A9_ABE = im[48:60]
A9_pUC19 = im[60:72]

In [ ]:
# i=5 for A8_pUC19

i = 0
condition = A8_ABE
af488 = condition[i][0]
af561 = condition[i][1]
af594 = condition[i][2]
af633 = condition[i][3]
cfp = condition[i][4]
dapi = condition[i][5]

In [ ]:
af561 = register_images(af488, af561)
af594 = register_images(af488, af594)
af633 = register_images(af488, af633)

In [ ]:
def imadjust(img, lower_bound=0.05, upper_bound=99.95):
    lower = np.percentile(img, lower_bound)
    upper = np.percentile(img, upper_bound)
    out = (img - lower) * (255 / (upper - lower))
    return np.clip(out, 0, 255, out)

In [ ]:
af488_adj = imadjust(af488)
af561_adj = imadjust(af561)
af594_adj = imadjust(af594)
af633_adj = imadjust(af633)

In [ ]:
af488_g, af561_g, af594_g = filt_gauss(af488_adj, af561_adj, af594_adj, 1.5)
af633_g, cfp_g, dapi_g = filt_gauss(af633_adj, cfp, dapi, 1.5)

In [ ]:
threshold = threshold_otsu(dapi_g)
print(threshold, 'is where the cutoff point is.')
dapi_bw = dapi_g > threshold

In [ ]:
dapi_labels, df_dapi, dapi_area, dapi_distance, dapi_markers = ws(dapi_bw, 2, np.ones((45,45)), 2000, dapi)

In [ ]:
bokeh.io.show(show_three_ims(dapi, dapi_bw, dapi_labels,
                             cmap=[None, None, colorcet.b_glasbey_hv]))


Next I will apply a LoG filter. This filter detects edges, which are defined by areas of sudden peaks or valleys in the gradient (first derivative) of the pixel intensity values. A peak or a valley in the first derivative means there is a zero-crossing in the second derivative.

In [ ]:
af488_edge_zero, af488_LoG, af488_edge = log_filter(af488_g, 1.5, 4, 1)
af561_edge_zero, af561_LoG, af561_edge = log_filter(af561_g, 1.5, 4, 1)
af594_edge_zero, af594_LoG, af594_edge = log_filter(af594_g, 1.5, 4, 1)
af633_edge_zero, af633_LoG, af633_edge = log_filter(af633_g, 1.5, 4, 1)

In [ ]:
# Show result
'''
bokeh.io.show(show_four_ims(af488_g, af488_LoG, af488_edge, af488_edge_zero,
                            titles=["488", "LoG", "edge", "zero"]))
'''

In [ ]:
# Show result
bokeh.io.show(show_four_ims(af488_g, af488_edge_zero, af594_g, af594_edge_zero,
                            titles=["488", "488", "594", "594"]))

In [ ]:
# Show result
bokeh.io.show(show_four_ims(af594_g, af594_edge_zero, af633_g, af633_edge_zero,
                            titles=["561", "561", "633", "633"]))

## Step 2. Segmentation

Next, I will skeletonize to get single pixel edges.

In [ ]:
# Skeletonize edges
af488_edge_sk = skimage.morphology.skeletonize(af488_edge_zero)
af561_edge_sk = skimage.morphology.skeletonize(af561_edge_zero)
af594_edge_sk = skimage.morphology.skeletonize(af594_edge_zero)
af633_edge_sk = skimage.morphology.skeletonize(af633_edge_zero)

# See result
bokeh.io.show(show_two_ims(af488_g, af488_edge_sk, titles=["original", "edges"]))

I will fill the holes created from the edges. Note that this will not fill holes that have any openings in them, even if the openings are very small.

In [ ]:
# Fill holes
af488_bw = ndi.morphology.binary_fill_holes(af488_edge_sk)
af561_bw = ndi.morphology.binary_fill_holes(af561_edge_sk)
af594_bw = ndi.morphology.binary_fill_holes(af594_edge_sk)
af633_bw = ndi.morphology.binary_fill_holes(af633_edge_sk)

In [ ]:
# Show result
bokeh.io.show(show_four_ims(af488_g, af488_bw, af594_g, af594_bw,
                            titles=["488", "488", "594", "594"]))

I will remove small objects. Note that this removes any objects that are not filled as well and will remove dots if they are not already filled.

In [ ]:
# Remove small objectes that are not bacteria
af488_bw = skimage.morphology.remove_small_objects(af488_bw, min_size=10)
af561_bw = skimage.morphology.remove_small_objects(af561_bw, min_size=10)
af594_bw = skimage.morphology.remove_small_objects(af594_bw, min_size=10)
af633_bw = skimage.morphology.remove_small_objects(af633_bw, min_size=10)

In [ ]:
af488_labels, df_af488, af488_area, af488_distance, af488_markers = ws(af488_bw, 1, np.ones((8,8)), 10, af488)
af561_labels, df_af561, af561_area, af561_distance, af561_markers = ws(af561_bw, 1, np.ones((8,8)), 10, af561)
af594_labels, df_af594, af594_area, af594_distance, af594_markers = ws(af594_bw, 1, np.ones((8,8)), 10, af594)
af633_labels, df_af633, af633_area, af633_distance, af633_markers = ws(af633_bw, 1, np.ones((8,8)), 10, af633)

In [ ]:
bokeh.io.show(show_four_ims(af488_g, af488_labels, af594_g, af594_labels,
                            titles=["488", "488", "594", "594"],
                            cmap=[None, colorcet.b_glasbey_hv, None, colorcet.b_glasbey_hv]))

The watershed works fine but results in irregularly shaped dots. I will address this by using the binary_opening tool, followed by erosion, and a relabel step. This will result in eroded dots that have a more typical circular shape.

In [ ]:
selem = skimage.morphology.disk(1)
af488_open = skimage.morphology.binary_opening(af488_labels, selem)
af561_open = skimage.morphology.binary_opening(af561_labels, selem)
af594_open = skimage.morphology.binary_opening(af594_labels, selem)
af633_open = skimage.morphology.binary_opening(af633_labels, selem)

In [ ]:
selem = skimage.morphology.disk(1)
af488_e = label(skimage.morphology.erosion(af488_open, selem))
af561_e = label(skimage.morphology.erosion(af561_open, selem))
af594_e = label(skimage.morphology.erosion(af594_open, selem))
af633_e = label(skimage.morphology.erosion(af633_open, selem))

In [ ]:
bokeh.io.show(show_six_ims(af488_g, af488_labels, af488_e, af594_g, af594_labels, af594_e,
                            titles=["488", "488", "488", "594", "594", "594"],
                            cmap=[None, colorcet.b_glasbey_hv, colorcet.b_glasbey_hv,
                                  None, colorcet.b_glasbey_hv, colorcet.b_glasbey_hv]))

Next I will create an OR mask. I will relabel this as well, as the logical_or function results in a binary image.

In [ ]:
af488_594 = label(np.logical_or(af488_e, af594_e))
af561_633 = label(np.logical_or(af561_e, af633_e))

bokeh.io.show(show_five_ims(af488_e, af594_e, af488_594, af488_g, af594_g,
                             titles=['488', '594', 'OR', '488', '594'], 
                             cmap=[colorcet.b_glasbey_hv, colorcet.b_glasbey_hv, colorcet.b_glasbey_hv, None, None]))

We can see that the OR statement results in some dots that overlap. I will plot a histogram to look at the typical dot sizes in the OR mask so that I can choose a maximum size threshold.

In [ ]:
props = skimage.measure.regionprops_table(af488_594, properties=('label',
                                                                 'centroid',
                                                                 'area',
                                                                 ))
df = pd.DataFrame(props)
df

In [ ]:
hv.Histogram(data=np.histogram(df['area'], bins=1000),
             kdims=['area'])

From this histogram, I can see that most of the dot sizes are less than 75 (total area). I will use this as my maximum size threshold.

In [ ]:
df.loc[df['area'] < 75]

In [ ]:
# define a function to remove large objects
def remove_large(label_image, max_size, df):
    '''
    Function to remove large segments from image.
    
    ----------
    Parameters
    ----------
    label_image : array
        watershed image with labels
    max_size : int
        remove all segments larger than this size. Default is 1000
    df : DataFrame
        DataFrame of watershed image
    
    --------
    Output
    --------
    large_sub : array
        image with large segments subtracted
    df_large : DataFrame
        DataFrame with large segments subtracted
        
    '''
    if label_image.sum() != 0:
        empty_array = np.zeros_like(label_image)
        max_size = max_size

        for i, label in enumerate(df.loc[df.loc[:, 'area'] > max_size]['label']):
            x = label_image == label
            y = x * label
            empty_array += y

        large_sub = label_image - empty_array
    return large_sub

In [ ]:
af488_594_largesub = remove_large(af488_594, 75, df)
af561_633_largesub = remove_large(af561_633, 75, df)

In [ ]:
af488_594_smallsub = skimage.morphology.remove_small_objects(af488_594_largesub, min_size=4)
af561_633_smallsub = skimage.morphology.remove_small_objects(af561_633_largesub, min_size=4)

In [ ]:

bokeh.io.show(show_four_ims(af488_594, af488_594_smallsub, af488_g, af594_g,
                             titles=['OR', 'OR large sub', '488', '594'], 
                             cmap=[colorcet.b_glasbey_hv, colorcet.b_glasbey_hv, None, None]))


In [ ]:
af488_594_filt = af488_594_smallsub * dapi_bw
af561_633_filt = af561_633_smallsub * dapi_bw

In [ ]:
'''
bokeh.io.show(show_two_ims(af488_594_largesub, af488_594_smallsub,
                             titles=['OR large sub', 'OR large and small sub'], 
                             cmap=[colorcet.b_glasbey_hv, colorcet.b_glasbey_hv]))
                             '''

In [ ]:
def coloc(large_sub, unedited, edited):
    '''
    Function to get all nuclei with mean intensity for each channel. Can do two channels.
    
    ---------
    Parameters
    ---------
    large_sub : array
        OR watershed image with small and large segments subtracted
    unedited : array
        raw intensity image
    edited : array
        raw intensity image
    
    -------
    Output
    -------
    df_co : DataFrame
        DataFrame with all dots with mean intensity for each channel.
    '''

    props_unedited = skimage.measure.regionprops_table(large_sub, intensity_image=unedited, properties=('label',
                                                                                             'centroid',
                                                                                             'area',
                                                                                             'mean_intensity'))
    df_unedited = pd.DataFrame(props_unedited)
    df_unedited['sum_intensity'] = df_unedited['area']*df_unedited['mean_intensity']
    

    props_edited = skimage.measure.regionprops_table(large_sub, intensity_image=edited, properties=('label',
                                                                                                 'centroid',
                                                                                                 'area',
                                                                                                 'mean_intensity'))
    df_edited = pd.DataFrame(props_edited)
    df_edited['sum_intensity'] = df_edited['area']*df_edited['mean_intensity']
    
    mean_unedited_vals = df_unedited['mean_intensity'].values
    mean_edited_vals = df_edited['mean_intensity'].values
    
    sum_unedited_vals = df_unedited['sum_intensity'].values
    sum_edited_vals = df_edited['sum_intensity'].values
    
    data = {'Unedited mean intensity': mean_unedited_vals, 
            'Edited mean intensity': mean_edited_vals,
            'Unedited sum intensity': sum_unedited_vals,
            'Edited sum intensity': sum_edited_vals}
    df_co = pd.DataFrame(data)
    return df_co, df_unedited, df_edited

In [ ]:
df_bc1, df_bc1_unedited, df_bc1_edited = coloc(af488_594_filt, af488, af594)
df_bc2, df_bc2_unedited, df_bc2_edited = coloc(af561_633_filt, af561, af633)

In [ ]:
df_bc1_unedited

In [ ]:
df_bc1['ln(intensity) of unedited probe'] = np.log(df_bc1['Unedited sum intensity'])
df_bc1['ln(intensity) of edited probe'] = np.log(df_bc1['Edited sum intensity'])
df_bc2['ln(intensity) of unedited probe'] = np.log(df_bc2['Unedited sum intensity'])
df_bc2['ln(intensity) of edited probe'] = np.log(df_bc2['Edited sum intensity'])

In [ ]:
df_bc1

In [ ]:
hv.Points(data=df_bc1,
          kdims=['Unedited mean intensity', 'Edited mean intensity'],
          label='bc1'
).opts(
    logy=True, 
    logx=True
)

In [ ]:
hv.Points(data=df_bc1,
          kdims=['ln(intensity) of unedited probe', 'ln(intensity) of edited probe'],
          label='Label'
)

In [ ]:
hv.Points(data=df_bc2,
          kdims=['Unedited mean intensity', 'Edited mean intensity'],
          label='bc2'
).opts(
    logy=True, 
    logx=True
)

In [ ]:
hv.Points(data=df_bc2,
          kdims=['Unedited sum intensity', 'Edited sum intensity'],
          label='Label'
)